In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.basics import *
import json

import jkbc.model as m
import jkbc.model.factory as factory
import jkbc.utils.constants as constants
import jkbc.utils.torch_files as f
import jkbc.utils.preprocessing as prep
import jkbc.utils.postprocessing as pop
import jkbc.utils.fasta as fasta

import jkbc.utils.kd as kd

## Constants

### Data

In [3]:
BASE_DIR = Path("../..")
PATH_DATA = 'data/feather-files'
DATA_SET = 'Range0-50-FixLabelLen400-winsize4096'
FEATHER_FOLDER = BASE_DIR/PATH_DATA/DATA_SET

with open(FEATHER_FOLDER/'config.json', 'r') as fp:
    config = json.load(fp)

ALPHABET       = constants.ALPHABET
ALPHABET_VAL   = list(ALPHABET.values())
ALPHABET_STR   = ''.join(ALPHABET_VAL)
ALPHABET_SIZE  = len(ALPHABET.keys())
WINDOW_SIZE    = int(config['maxw']) #maxw = max windowsize
DIMENSIONS_OUT = int(config['maxl']) # maxl = max label length
MIN_LABEL_LEN  = int(config['minl']) # maxl = max label length
STRIDE         = WINDOW_SIZE

### Train/Predict

In [4]:
BS = 2**3  # batch size
DEVICE = torch.device("cuda:0") #torch.device("cpu")

### Model

In [ ]:
model, (MODEL_NAME, _) = factory.bonito(WINDOW_SIZE, DEVICE, BASE_DIR)
MODEL_NAME = f'{MODEL_NAME}'
MODEL_DIR = f'{MODEL_NAME}/weights'
model_weights = 'bestmodel_8'
predicter = m.get_predicter(model, MODEL_DIR, DEVICE)

In [ ]:
if not model_weights:
    model_weights = m.get_newest_model(MODEL_DIR)

predicter.load(model_weights);

## Save teacher output

In [ ]:
# Read data from feather
data = f.load_training_data(FEATHER_FOLDER) 
kd.generate_and_save_y_teacher(FEATHER_FOLDER, MODEL_NAME, m.signal_to_input_tensor(data.x, DEVICE), predicter.model, bs=BS)

## Get accuracies

In [ ]:
sc = prep.SignalCollection(BASE_DIR/constants.MAPPED_READS, labels_per_window=(MIN_LABEL_LEN,DIMENSIONS_OUT), 
                           stride=STRIDE, window_size=(WINDOW_SIZE-1, WINDOW_SIZE), blank_id=constants.BLANK_ID)

In [ ]:
# predict range
predict_objects = m.predict_range(predicter.model, sc, ALPHABET_STR, WINDOW_SIZE, DEVICE, indexes=np.random.randint(0, len(sc), 50))

In [ ]:
# convert outputs to dictionaries
references = []
predictions = []
accuracies = []
for po in predict_objects:
    [accuracies.append(a) for a in m.get_accuracies(po.references, po.predictions, ALPHABET_VAL)]
    ref, pred = fasta.map_decoded(po, ALPHABET_VAL, False)
    references.append(ref)
    predictions.append(pred)

# Sanity tjek
np.mean(accuracies)

In [ ]:
# make dicts ready to be saved
ref_dict = fasta.merge(references)
pred_dict = fasta.merge(predictions)

In [ ]:
# save dicts
fasta.save_dicts(pred_dict, ref_dict, f'{MODEL_NAME}/predictions/{DATA_SET}')